Detección de caras con webcam

In [1]:
import cv2
import time
import FaceNormalizationUtils as faceutils
# My face detectors interface
import FaceDetectors
import math
import numpy as np

def rotate_from_origin(xy, radians):
    """Only rotate a point around the origin (0, 0)."""
    x, y = xy
    xx = x * math.cos(radians) + y * math.sin(radians)
    yy = -x * math.sin(radians) + y * math.cos(radians)

    return xx, yy

2023-10-30 21:09:01.976130: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-30 21:09:02.380370: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-30 21:09:02.380481: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-30 21:09:02.382553: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-30 21:09:02.609611: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-30 21:09:02.611187: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

En el bucle de procesamiento, las teclas 'd' y 'e' `permiten respectivamenet cambiar de modelo de detección de caras, y en su caso de máscara de detección del rostro.

La ejecución de la siguiente celda produce error al no disponer de los archivos shape_predictor_5_face_landmarks.dat y shape_predictor_68_face_landmarks.dat que por su tamaño no se incluyeron en el repositorio. Pueden descargarse desde el enlace proporcionado en el campus virtual (opción aconsejada), o
desde el [repositorio de archivos de dlib](http://dlib.net/files/).



In [36]:
normalizatorHS = faceutils.Normalization()

# Face detectors interface
FDet = FaceDetectors.FaceDetector()

faceD = FDet.FaceDetectors[1]
eyesD = FDet.EyeDetectors[1]

size = 4
historical = [[0 for j in range(size)] for i in range(3)]
pointer = 0

def put_xyz_ang(x_ang, y_ang, z_ang):
    global historical
    global pointer
    global size
    thresh = 0.35
        
    historical[0][pointer] = x_ang if abs(y_ang) < thresh else historical[0][(pointer+size-1)%size]
    historical[1][pointer] = y_ang
    historical[2][pointer] = z_ang
    pointer += 1
    pointer %= size

def get_xyz_ang():
    global historical
    return [sum(historical[i]) for i in range(3)]

laser_diameter = 10
laser_length = 200
laser_color = (0, 0, 255)
compensation_x_ang = 5
# Fonts
font = cv2.FONT_HERSHEY_SIMPLEX

# Webcam connection
cap = cv2.VideoCapture(0)
# Check for other cameras
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print('Camera error')
            exit(0)
        else:
            print('Camera 0')
    else:
        print('Camera 1')
else:
    print('Camera 0')

debug = False

#Set camera resolution
cap.set(3,640)
cap.set(4,480)

while True:
    # Get frame
    t = time.time()
    ret, frame = cap.read()
    # For HS normalization
    B, G, R = cv2.split(frame)

    # Search face with a specific setup for face and eye detection
    values = FDet.SingleFaceEyesDetection(frame, faceD, eyesD)
    if values is not None:
        face, eyes, shape = values

        # Ángulo positivo = girar en sentido antihorario
        # x = mirar abajo, y = mirar izquierda, z = girar derecha
        # Unidad = radianes
        x_ang = y_ang = z_ang = 0   



        #draws face container
        [x, y , w, h] = face
        if x > -1:
            #cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

            # draws eyes and mask if available
            [lex, ley, rex, rey] = eyes
            if lex > -1:
                # Show detected facial elements
                if debug:
                    for (x, y) in shape:
                        cv2.circle(frame, (x, y), 2, (255, 255, 255), -1)

                    for i in (33, 2, 14):
                        cv2.circle(frame, (shape[i,0], shape[i,1]), 2, (255, 0, 0), -1)

                
                ab = (shape[14]-shape[2])
                ap = (shape[33]-shape[2])
                distCheeks = np.linalg.norm(ab)
                distNose = np.linalg.norm(np.dot(ap, ab) / np.dot(ab, ab) * ab)

                z_ang = math.atan((shape[26,1] - shape[17,1]) / (shape[26,0] - shape[17,0]))
                y_ang = prop = distNose / distCheeks - 0.5
                x_ang = rotate_from_origin((lex, ley), z_ang)[1]-rotate_from_origin(shape[0], z_ang)[1] \
                if y_ang > 0 else rotate_from_origin((rex, rey), z_ang)[1]-rotate_from_origin(shape[16], z_ang)[1] - compensation_x_ang
                x_ang *= (1+abs(y_ang)) / 120

                put_xyz_ang(x_ang, y_ang, z_ang)

                if debug:
                    if y_ang > 0:
                        cv2.circle(frame, ((int)(lex), (int)(ley)), 4, (0, 0, 255), -1)
                    else:
                        cv2.circle(frame, ((int)(rex), (int)(rey)), 4, (0, 255, 0), -1)
                    print(x_ang, y_ang, z_ang)

                x2, y2, z2 = get_xyz_ang()
                # TRABAJAR AQUÍ
                if not debug:
                    looking_pt = rotate_from_origin((math.sin(y2)*laser_length, math.sin(x2)*laser_length), -z2)
                    looking_pt_l = (int(looking_pt[0]+lex), int(looking_pt[1]+ley))
                    looking_pt_r = (int(looking_pt[0]+rex), int(looking_pt[1]+rey))

                    cv2.line(frame, (int(lex), int(ley)), looking_pt_l, laser_color, laser_diameter)
                    cv2.line(frame, (int(rex), int(rey)), looking_pt_r, laser_color, laser_diameter)





    #if debug:
    #    print("Processing time : {:.3f}".format(time.time() - t))

    # Show resulting image
    cv2.putText(frame, faceD, (10, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, eyesD, (50, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.imshow('Cam', frame)
    
    # Esc to finish
    tec = cv2.waitKey(40)
    if tec & tec == 27:  # Esc
        break

# Close windoews and release camera
cap.release()
cv2.destroyAllWindows()

Camera 0
